In [1]:
import init
from SBMLLint.common import constants as cn
from SBMLLint.common.molecule import Molecule, MoleculeStoichiometry
from SBMLLint.common.reaction import Reaction
from SBMLLint.games.som import SOM
from SBMLLint.common.simple_sbml import SimpleSBML

from SBMLLint.common import config
from SBMLLint.common import constants as cn
from SBMLLint.common import util
from SBMLLint.games.games_pp import GAMES_PP
from SBMLLint.games.games_report import GAMESReport
from SBMLLint.structured_names.moiety_comparator import MoietyComparator

import argparse
import os
import sys
import tesbml

In [2]:
from SBMLLint.tools import sbmllint
import os
cwd = os.getcwd()
MAIN_DIR = os.path.join(os.getcwd(), os.pardir)
os.chdir(MAIN_DIR)
print(os.getcwd())

/Users/woosubs/Desktop/ModelEngineering/SBMLLint/SBMLLint


In [3]:
PREFIX = "BIOMD0000000"

def load_file_from_curated_data(num):

  format_num = format(num, '03d')
  filename = "%s%s_url.xml" % (PREFIX, format_num)
  path = os.path.join(cn.BIOMODELS_DIR, filename)
  simple = SimpleSBML()
  simple.initialize(path)
  return simple

In [14]:
os,

'/Users/woosubs/Desktop/ModelEngineering/SBMLLint/SBMLLint/SBMLLint/tests'

In [4]:
simple1 = load_file_from_curated_data(167)
simple2 = load_file_from_curated_data(5)
print("simple1:\n", simple1.reactions, "\n\n")
print("simpel2:\n", simple2.reactions)

simple1:
 [PstatDimerisation: 2.00 Pstat_sol -> PstatDimer_sol; (Kf_PstatDimerisation * pow(Pstat_sol, 2) + -(Kr_PstatDimerisation * PstatDimer_sol)) * sol, statDephosphorylation: Pstat_nuc -> stat_nuc; Kcat_dephos * statPhosphatase_nuc * Pstat_nuc * (1 / (Km_dephos + Pstat_nuc)) * nuc, statPhosphorylation: stat_sol -> Pstat_sol + species_test; Kcat_phos * statKinase_sol * stat_sol * (1 / (Km_phos + stat_sol)) * sol, PstatDimerisationNuc: 2.00 Pstat_nuc -> PstatDimer_nuc; (Kf_PstatDimerisation * pow(Pstat_nuc, 2) + -(Kr_PstatDimerisation * PstatDimer_nuc)) * nuc, PstatDimer__import: PstatDimer_sol -> PstatDimer_nuc; PstatDimer_impMax * PstatDimer_sol * (1 / (Kpsd_imp + PstatDimer_sol)) * nm, stat_export: stat_sol -> stat_nuc; nuc * stat_expMax * stat_nuc * (1 / (Ks_exp + stat_nuc)) * nm, stat_import: stat_sol -> stat_nuc; nuc * stat_impMax * stat_sol * (1 / (Ks_imp + stat_sol)) * nm] 


simpel2:
 [Reaction1: M -> C2 + YP; cell * k6 * M, Reaction2: C2 -> CP; cell * C2 * k8notP, Reaction

In [5]:
reaction = simple2.getReaction("Reaction1")
print(reaction)
print(reaction.label)

Reaction1: M -> C2 + YP; cell * k6 * M
Reaction1


In [6]:
reaction.__class__

SBMLLint.common.reaction.Reaction

In [7]:
simple1.add(reaction)
simple1.reactions

[PstatDimerisation: 2.00 Pstat_sol -> PstatDimer_sol; (Kf_PstatDimerisation * pow(Pstat_sol, 2) + -(Kr_PstatDimerisation * PstatDimer_sol)) * sol,
 statDephosphorylation: Pstat_nuc -> stat_nuc; Kcat_dephos * statPhosphatase_nuc * Pstat_nuc * (1 / (Km_dephos + Pstat_nuc)) * nuc,
 statPhosphorylation: stat_sol -> Pstat_sol + species_test; Kcat_phos * statKinase_sol * stat_sol * (1 / (Km_phos + stat_sol)) * sol,
 PstatDimerisationNuc: 2.00 Pstat_nuc -> PstatDimer_nuc; (Kf_PstatDimerisation * pow(Pstat_nuc, 2) + -(Kr_PstatDimerisation * PstatDimer_nuc)) * nuc,
 PstatDimer__import: PstatDimer_sol -> PstatDimer_nuc; PstatDimer_impMax * PstatDimer_sol * (1 / (Kpsd_imp + PstatDimer_sol)) * nm,
 stat_export: stat_sol -> stat_nuc; nuc * stat_expMax * stat_nuc * (1 / (Ks_exp + stat_nuc)) * nm,
 stat_import: stat_sol -> stat_nuc; nuc * stat_impMax * stat_sol * (1 / (Ks_imp + stat_sol)) * nm,
 Reaction1: M -> C2 + YP; cell * k6 * M]

In [8]:
################### Below is about removing a reaction #################
simple2.remove(reaction)
simple2.reactions

[Reaction2: C2 -> CP; cell * C2 * k8notP,
 Reaction3: CP -> C2; cell * CP * k9,
 Reaction4: CP + Y -> pM; cell * CP * k3 * Y,
 Reaction5: M -> pM; cell * k5notP * M,
 Reaction6: EmptySet -> Y; cell * k1aa,
 Reaction7: Y -> EmptySet; cell * k2 * Y,
 Reaction8: YP -> EmptySet; cell * k7 * YP,
 Reaction9: pM -> M; cell * pM * (k4prime + k4 * pow(M / CT, 2))]

In [9]:
################# Below is about modifying reactions for implicits #####################
r = simple1.getReaction("Reaction1")
print(r)

Reaction1: M -> C2 + YP; cell * k6 * M


In [13]:
implicit = "M"
print("Before:")
for r in simple2.reactions:
  print(r.makeIdentifier(is_include_kinetics=False))
modified_reactions = []
for r in simple2.reactions:
  print("Current Reaction % ", r.makeIdentifier(is_include_kinetics=False))
  reactant_names = [reactant.molecule.name for reactant in r.reactants]
  product_names = [product.molecule.name for product in r.products]
  if (implicit in reactant_names) or (implicit in product_names):
    print("This reaction has a removable implicit: %s" % implicit)
    if implicit in reactant_names:
      new_reactants = [ms for ms in r.reactants if ms.molecule.name != implicit]
    else:
      new_reactants = r.reactants
    if implicit in product_names:
      new_products = [ms for ms in r.products if ms.molecule.name != implicit]
    else:
      new_products = r.products
    r.reactants = new_reactants
    r.products = new_products
    r.identifier = r.makeIdentifier()
    r.category = r.getCategory()
    modified_reactions.append(r)
print("\n\n***************After**************")
for r in simple2.reactions:
  print(r.makeIdentifier(is_include_kinetics=False))

Before:
Reaction2: C2 -> CP
Reaction3: CP -> C2
Reaction4: CP + Y -> pM
Reaction5:  -> pM
Reaction6: EmptySet -> Y
Reaction7: Y -> EmptySet
Reaction8: YP -> EmptySet
Reaction9: pM -> 
Current Reaction %  Reaction2: C2 -> CP
Current Reaction %  Reaction3: CP -> C2
Current Reaction %  Reaction4: CP + Y -> pM
Current Reaction %  Reaction5:  -> pM
Current Reaction %  Reaction6: EmptySet -> Y
Current Reaction %  Reaction7: Y -> EmptySet
Current Reaction %  Reaction8: YP -> EmptySet
Current Reaction %  Reaction9: pM -> 


***************After**************
Reaction2: C2 -> CP
Reaction3: CP -> C2
Reaction4: CP + Y -> pM
Reaction5:  -> pM
Reaction6: EmptySet -> Y
Reaction7: Y -> EmptySet
Reaction8: YP -> EmptySet
Reaction9: pM -> 


In [11]:
r5 = simple2.getReaction("Reaction5")
r5.category

'reaction_boundary'

In [15]:
r.products = [MoleculeStoichiometry(molecule=r.products[0].molecule, stoichiometry=r.products[0].stoichiometry),
             MoleculeStoichiometry(molecule=r.reactants[0].molecule, stoichiometry=r.reactants[0].stoichiometry)]
r.reactants = []
r.identifier = r.makeIdentifier()
r

Reaction1:  -> C2 + M; cell * k6 * M

In [17]:
simple1.remove(simple1.getReaction("Reaction1"))
simple1.reactions

[PstatDimerisation: 2.00 Pstat_sol -> PstatDimer_sol; (Kf_PstatDimerisation * pow(Pstat_sol, 2) + -(Kr_PstatDimerisation * PstatDimer_sol)) * sol,
 statDephosphorylation: Pstat_nuc -> stat_nuc; Kcat_dephos * statPhosphatase_nuc * Pstat_nuc * (1 / (Km_dephos + Pstat_nuc)) * nuc,
 statPhosphorylation: stat_sol -> Pstat_sol + species_test; Kcat_phos * statKinase_sol * stat_sol * (1 / (Km_phos + stat_sol)) * sol,
 PstatDimerisationNuc: 2.00 Pstat_nuc -> PstatDimer_nuc; (Kf_PstatDimerisation * pow(Pstat_nuc, 2) + -(Kr_PstatDimerisation * PstatDimer_nuc)) * nuc,
 PstatDimer__import: PstatDimer_sol -> PstatDimer_nuc; PstatDimer_impMax * PstatDimer_sol * (1 / (Kpsd_imp + PstatDimer_sol)) * nm,
 stat_export: stat_sol -> stat_nuc; nuc * stat_expMax * stat_nuc * (1 / (Ks_exp + stat_nuc)) * nm,
 stat_import: stat_sol -> stat_nuc; nuc * stat_impMax * stat_sol * (1 / (Ks_imp + stat_sol)) * nm]

In [18]:
simple1.add(r)
simple1.reactions

[PstatDimerisation: 2.00 Pstat_sol -> PstatDimer_sol; (Kf_PstatDimerisation * pow(Pstat_sol, 2) + -(Kr_PstatDimerisation * PstatDimer_sol)) * sol,
 statDephosphorylation: Pstat_nuc -> stat_nuc; Kcat_dephos * statPhosphatase_nuc * Pstat_nuc * (1 / (Km_dephos + Pstat_nuc)) * nuc,
 statPhosphorylation: stat_sol -> Pstat_sol + species_test; Kcat_phos * statKinase_sol * stat_sol * (1 / (Km_phos + stat_sol)) * sol,
 PstatDimerisationNuc: 2.00 Pstat_nuc -> PstatDimer_nuc; (Kf_PstatDimerisation * pow(Pstat_nuc, 2) + -(Kr_PstatDimerisation * PstatDimer_nuc)) * nuc,
 PstatDimer__import: PstatDimer_sol -> PstatDimer_nuc; PstatDimer_impMax * PstatDimer_sol * (1 / (Kpsd_imp + PstatDimer_sol)) * nm,
 stat_export: stat_sol -> stat_nuc; nuc * stat_expMax * stat_nuc * (1 / (Ks_exp + stat_nuc)) * nm,
 stat_import: stat_sol -> stat_nuc; nuc * stat_impMax * stat_sol * (1 / (Ks_imp + stat_sol)) * nm,
 Reaction1:  -> C2 + M; cell * k6 * M]

In [105]:
result = []
collection = simple.reactions
print(collection)
for ele in collection:
  print("this is list", [not ele.isEqual(r) for r in result])
  if all([not ele.isEqual(r) for r in result]):
    result.append(ele)

[PstatDimerisation: 2.00 Pstat_sol -> PstatDimer_sol; (Kf_PstatDimerisation * pow(Pstat_sol, 2) + -(Kr_PstatDimerisation * PstatDimer_sol)) * sol, statDephosphorylation: Pstat_nuc -> stat_nuc; Kcat_dephos * statPhosphatase_nuc * Pstat_nuc * (1 / (Km_dephos + Pstat_nuc)) * nuc, statPhosphorylation: stat_sol -> Pstat_sol + species_test; Kcat_phos * statKinase_sol * stat_sol * (1 / (Km_phos + stat_sol)) * sol, PstatDimerisationNuc: 2.00 Pstat_nuc -> PstatDimer_nuc; (Kf_PstatDimerisation * pow(Pstat_nuc, 2) + -(Kr_PstatDimerisation * PstatDimer_nuc)) * nuc, PstatDimer__import: PstatDimer_sol -> PstatDimer_nuc; PstatDimer_impMax * PstatDimer_sol * (1 / (Kpsd_imp + PstatDimer_sol)) * nm, stat_export: stat_sol -> stat_nuc; nuc * stat_expMax * stat_nuc * (1 / (Ks_exp + stat_nuc)) * nm, stat_import: stat_sol -> stat_nuc; nuc * stat_impMax * stat_sol * (1 / (Ks_imp + stat_sol)) * nm]
this is list []
this is list [True]
this is list [True, True]
this is list [True, True, True]
this is list [True,

In [16]:
TYPE_I = "type1"
TYPE_II = "type2"
TYPE_III = "type3"
CANCELING = "canceling"
ECHELON = "echelon"


def lint(model_reference, file_out=sys.stdout,
    mass_balance_check="structured_names",
    config_path=None,
    is_report=True):
  """
  Reports on errors found in a model
  :param str model_reference: 
      libsbml_model, file, antimony string, xml string
  :param TextIOWrapper file_out:
  :param str mass_balance_check: how check for mass balance
  :param str config_path: path to configuration file
  :param bool is_report: print result
  :return MoietyComparatorResult/bull/None:
  """
  config.setConfiguration(path=config_path)
  config_dict = config.getConfiguration()
  if util.isSBMLModel(model_reference):
    model = model_reference
  else:
    xml = util.getXML(model_reference)
    reader = tesbml.SBMLReader()
    document = reader.readSBMLFromString(xml)
    util.checkSBMLDocument(document)
    model = document.getModel()
  simple = SimpleSBML()
  simple.initialize(model)
  if mass_balance_check == "structured_names":
    result = MoietyComparator.analyzeReactions(simple,
        implicits=config_dict['implicits'])
    if is_report:
      for line in result.report.split('\n'):
          file_out.write("%s\n" % line)
    return result
  elif mass_balance_check == "games":
    m = GAMES_PP(simple)
    games_result = m.analyze(simple.reactions)
    if games_result and is_report:
      gr = GAMESReport(m)
      errortype_dic = {TYPE_I: gr.reportTypeOneError,
                       TYPE_II: gr.reportTypeTwoError,
                       TYPE_III: gr.reportTypeThreeError,
                       CANCELING: gr.reportCancelingError,
                       ECHELON: gr.reportEchelonError
                      }
      for errors in m.error_summary:
        for category in errortype_dic.keys():
          if errors.type == category:
            func = errortype_dic[category]            
            report, _ = func(errors.errors, explain_details=True)
            print(report)
    print("By the way, config_dict is", config_dict['implicits'])
    return games_result
  else:
    print ("Specified method doesn't exist")
    return None

In [18]:
model2 = """
A -> B; 1
B -> A + C; 1
A = 0.5
B = 1.0
C = 0
"""
_ = lint(model2, mass_balance_check="games")

Model analyzed...
At least one error found.

A = B by reaction(s):
1. _J0: A -> B

However, A < B by reaction(s):
2. _J1: B -> A + C


----------------------------------------------------------------------



**********************************************************************


By the way, config_dict is ['DUMMYIMPLICIT']
